In [ ]:
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:

df = messages = pd.read_csv("hl7_data_small.csv")

df.head()

,Unnamed: 0,Unnamed: 0.1,id,message,date_time,patient_id,ecg,pulse_oximeter,bp,gsr,emg,labels
0,695,695,696,MSH|^~\&|EpiOne_ESP32|Personal_Device|Epione A...,22-09-2021 13:08,12,1,1,1,1,1,"[1, 1, 1]"
1,696,696,697,MSH|^~\&|EpiOne_ESP32|Personal_Device|Epione A...,22-09-2021 13:08,12,1,1,1,1,1,"[1, 1, 1]"
2,697,697,698,MSH|^~\&|EpiOne_ESP32|Personal_Device|Epione A...,22-09-2021 13:08,12,1,1,1,1,1,"[1, 1, 1]"
3,698,698,699,MSH|^~\&|EpiOne_ESP32|Personal_Device|Epione A...,22-09-2021 13:08,12,1,1,1,1,1,"[1, 1, 1]"
4,701,701,702,MSH|^~\&|EpiOne_ESP32|Personal_Device|Epione A...,22-09-2021 13:08,12,1,1,1,1,1,"[1, 1, 1]"


In [ ]:
X = df["message"]
y = df["ecg"]

In [ ]:
messages['message'][1]

'MSH|^~\\&|EpiOne_ESP32|Personal_Device|Epione Android App|Personal|||ORU^R01|59689|P|2.3|||AL|NE|#OBX|1|NM|ecg^ECG^Local^ecg^ECG^SDN||4095||||||F||||epionev2|v2^epione|#OBX|2|NM|pulse^Pulse^Local^pulse^Pulse^SDN||57||||||F||||epionev2|v2^epione|#OBX|3|NM|o2^O2^Local^o2^O2^SDN||100||||||F||||epionev2|v2^epione|#OBX|4|NM|temp^Temperature^Local^temp^Temperature^SDN||80.94||||||F||||epionev2|v2^epione|#OBX|5|NM|gsr^GSR^Local^gsr^GSR^SDN||1840||||||F||||epionev2|v2^epione|#OBX|6|NM|emg^EMG^Local^emg^EMG^SDN||784||||||F||||epionev2|v2^epione|#OBX|7|NM|bp_pulse^Sys_Dia_Pulse^Local^bp_pulse^Sys_Dia_Pulse^SDN||117, 080, 057||||||F||||epionev2|v2^epione|'

In [ ]:
messages.reset_index(inplace=True)


In [ ]:
corpus = []

for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z0-9]', ' ', messages['message'][i])
    review = re.sub(' +', ' ', review)
    review = review.lower()
    corpus.append(review)

print(i)

corpus[0]

791


'msh epione esp32 personal device epione android app personal oru r01 59689 p 2 3 al ne obx 1 nm ecg ecg local ecg ecg sdn 4095 f epionev2 v2 epione obx 2 nm pulse pulse local pulse pulse sdn 57 f epionev2 v2 epione obx 3 nm o2 o2 local o2 o2 sdn 100 f epionev2 v2 epione obx 4 nm temp temperature local temp temperature sdn 81 05 f epionev2 v2 epione obx 5 nm gsr gsr local gsr gsr sdn 1840 f epionev2 v2 epione obx 6 nm emg emg local emg emg sdn 795 f epionev2 v2 epione obx 7 nm bp pulse sys dia pulse local bp pulse sys dia pulse sdn 117 080 057 f epionev2 v2 epione'

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(y_train)
Test_Y = Encoder.fit_transform(y_test)

Train_X = X_train
Test_X = X_test

Train_Y[0:10]

array([1, 1, 0, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(corpus)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=18)

knn.fit(Train_X_Tfidf,Train_Y)

predicted = knn.predict(Test_X_Tfidf)

print("KNN Accuracy Score -> ",accuracy_score(predicted, Test_Y)*100)

KNN Accuracy Score ->  73.52941176470588
